In [1]:
import pandas as pd
import numpy as np

In [2]:
shot_probs_player_one = pd.read_csv('shot_probs_djokovic.csv')
sequence_probs_player_one = pd.read_csv('sequence_probs_djokovic.csv')
return_probs_player_one = pd.read_csv('return_probs_djokovic.csv')
return_sequence_player_one = pd.read_csv('return_sequence_prob_djokovic.csv')
first_serve_probabilities_player_one = pd.read_csv('first_serve_probabilities_novak.csv')
second_serve_probabilities_player_one = pd.read_csv('second_serve_probabilities_novak.csv')

shot_probs_player_two = pd.read_csv('shot_probs_sinner.csv')
sequence_probs_player_two = pd.read_csv('sequence_probs_sinner.csv')
return_probs_player_two = pd.read_csv('return_probs_sinner.csv')
return_sequence_prob_player_two = pd.read_csv('return_sequence_prob_sinner.csv')
first_serve_probabilities_player_two= pd.read_csv('first_serve_probabilities_sinner.csv')
second_serve_probabilities_player_two= pd.read_csv('second_serve_probabilities_sinner.csv')

In [3]:
sequence_probs_player_one = sequence_probs_player_one[~sequence_probs_player_one['shot_sequence'].str.endswith(('@', '#', '*'))]
sequence_probs_player_two = sequence_probs_player_two[~sequence_probs_player_two['shot_sequence'].str.endswith(('@', '#', '*'))]

In [49]:
precedent_shots_set = set(shot_probs_player_one['precedent_shot'].unique())
sequence_probs_player_two = sequence_probs_player_two[sequence_probs_player_two['shot_sequence'].apply(lambda x: x in precedent_shots_set)]

precedent_shots_set = set(shot_probs_player_two['precedent_shot'].unique())
sequence_probs_player_one = sequence_probs_player_one[sequence_probs_player_one['shot_sequence'].apply(lambda x: x in precedent_shots_set)]


In [50]:
def simulate_first_serve(player_serve_probs):
    
    service_types = player_serve_probs['serve_type'].tolist()
    service_probabilities = player_serve_probs['serve_type_probability'].tolist()
    service_probabilities = [float(i)/sum(service_probabilities) for i in service_probabilities]
    service_type = np.random.choice(service_types, p= service_probabilities)
    
    outcomes = ['ace_or_forcing_error', 'fault', 'in_game']
    outcome_probs = []
    for outcome in outcomes:
        outcome_probs.append(player_serve_probs[player_serve_probs['serve_type'] == service_type][outcome].iloc[0])
    serve_outcome = np.random.choice(outcomes, p= outcome_probs)
    
    return service_type, serve_outcome

In [51]:
def simulate_second_serve(player_serve_probs):
    
    service_types = player_serve_probs['serve_type'].tolist()
    service_probabilities = player_serve_probs['serve_type_probability'].tolist()
    service_probabilities = [float(i)/sum(service_probabilities) for i in service_probabilities]
    service_type = np.random.choice(service_types, p= service_probabilities)
    
    outcomes = ['ace_or_forcing_error', 'fault', 'in_game']
    outcome_probs = []
    for outcome in outcomes:
        outcome_probs.append(player_serve_probs[player_serve_probs['serve_type'] == service_type][outcome].iloc[0])
    serve_outcome = np.random.choice(outcomes, p= outcome_probs)
    
    return service_type, serve_outcome

In [52]:
def simulate_return(player_return_probs, player_return_sequence, serve):
    filtered_probs = player_return_probs[player_return_probs['preceding_serve'] == serve]
    shot_types = filtered_probs['shot_type'].tolist()
    happening_probabilities = filtered_probs['happening_probability'].tolist()
    return_type = np.random.choice(shot_types, p= happening_probabilities)
    
    outcomes = ['point_winner_prob', 'unforced_error_prob', 'in_play_prob']
    outcome_probs = []
    for outcome in outcomes:
        outcome_probs.append(filtered_probs[filtered_probs['shot_type'] == return_type][outcome].iloc[0])
    return_outcome = np.random.choice(outcomes, p= outcome_probs)
    
    if return_outcome == 'point_winner_prob' or return_outcome == 'unforced_error_prob':
        return_sequence = return_type
    else:
        filtered_return_sequence = player_return_sequence[player_return_sequence['shot_type'] == return_type]
        sequences =  filtered_return_sequence['return_shot_sequence'].tolist()
        sequence_probabilities = filtered_return_sequence['probability'].tolist()
        return_sequence = np.random.choice(sequences, p= sequence_probabilities)
        
    return return_sequence, return_outcome

In [170]:
def simulate_shot(precedent_shot, shot_probs, sequence_probs):
    filtered_probs = shot_probs[shot_probs['precedent_shot'] == precedent_shot]
    
    # Extract shot types and their happening probabilities
    shot_types = filtered_probs['shot_type'].tolist()
    happening_probabilities = filtered_probs['happening_probability'].tolist()
    
    # Check if there are shot types available for the precedent shot
    if shot_types:
        # Normalize happening_probabilities to ensure they sum to 1
        happening_probabilities_normalized = [float(i)/sum(happening_probabilities) for i in happening_probabilities]
        selected_shot_type = np.random.choice(shot_types, p=happening_probabilities_normalized)
        
        # Further processing to determine the outcome and next shot...
        outcome_probabilities = filtered_probs[['point_winner', 'unforced_error', 'in_play']].iloc[0].tolist()
        outcomes = ['point_winner', 'unforced_error', 'in_play']
        outcome_probabilities_normalized = [float(i)/sum(outcome_probabilities) for i in outcome_probabilities]
        selected_outcome = np.random.choice(outcomes, p=outcome_probabilities_normalized)
        
        if selected_outcome == 'in_play':
            filtered_sequence = sequence_probs[sequence_probs['shot_type'] == selected_shot_type]
            sequence_types = filtered_sequence['shot_sequence'].tolist()
            sequence_probabilities = filtered_sequence['probability'].tolist()
            sequence_probabilities_normalized = [float(i)/sum(sequence_probabilities) for i in sequence_probabilities]
            sequence_outcome = np.random.choice(sequence_types, p=sequence_probabilities_normalized)
        else:
            sequence_outcome = 'no_sequence'
    else:
        # Handle the case where no matching shot types are found
        selected_shot_type = 'no_shot'
        selected_outcome = 'no_outcome'
        sequence_outcome = 'no_sequence'
    
    return selected_outcome, sequence_outcome, selected_shot_type

        
        
    


In [169]:
def simulate_point(serve_probs, second_serve_probs, return_probs, return_sequence_probs, shot_probs_player1, sequence_probs_player1, shot_probs_player2, sequence_probs_player2):
    # Player One serves
    serve_type, serve_outcome = simulate_first_serve(serve_probs)
    rally_shot_count = 1
    if serve_outcome == 'fault':
        # Simulate second serve if the first serve was a fault
        serve_type, serve_outcome = simulate_second_serve(second_serve_probs)
        
        # If second serve also results in fault, Player Two wins the point
        if serve_outcome == 'fault':
            return 'returner', rally_shot_count

    if serve_outcome == 'ace_or_forcing_error':
        # Player One wins the point directly from the serve
        return 'server', rally_shot_count
    
    return_sequence, return_outcome = simulate_return(return_probs, return_sequence_probs, serve_type)
    
    if return_outcome == 'point_winner_prob':
        rally_shot_count += 1
        return 'returner', rally_shot_count
    elif return_outcome == 'unforced_error_prob':
        rally_shot_count += 1
        return 'server', rally_shot_count
    
    rally_shot_count = 3
    current_shot = return_sequence
    
    while True:
        player_shot_probs = shot_probs_player1 if rally_shot_count % 2 == 1 else shot_probs_player2
        player_sequence_probs = sequence_probs_player1 if rally_shot_count % 2 == 1 else sequence_probs_player2
        
        outcome, next_shot, shot_type = simulate_shot(current_shot, player_shot_probs, player_sequence_probs)

        if shot_type == 'no_shot':

            winner = 'server' if rally_shot_count % 2 == 0 else 'returner'  
            return winner, rally_shot_count

        if outcome == 'point_winner':
            winner = 'server' if rally_shot_count % 2 == 1 else 'returner'
            return winner, rally_shot_count
        elif outcome == 'unforced_error':
            winner = 'returner' if rally_shot_count % 2 == 1 else 'server'
            return winner, rally_shot_count

        current_shot = next_shot
        rally_shot_count += 1



In [205]:
server=0
returner=0
total_count = 0
for i in range(1000):
    outcome, count = simulate_point(first_serve_probabilities_player_two, second_serve_probabilities_player_two, return_probs_player_one, return_sequence_player_one, shot_probs_player_two, sequence_probs_player_two, shot_probs_player_one, sequence_probs_player_one)
    total_count += count
    if outcome == 'server':
        server +=1
    elif outcome == 'returner':
        returner +=1
    else:
        print('error')
print(server/(server + returner)*100)
print(total_count/1000)      


38.6
5.189
